# study goals



In [2]:
!pip install -qU langchain-ollama langgraph
%pip install -qU duckduckgo-search langchain-community
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately
recurse = 50 # model recursion limit, update if needed.
llm = ChatOllama(model="qwen3:1.7b")
search = DuckDuckGoSearchRun()
privateState = {}

def addState(keysTup: tuple, valStr: str):
    """
    store useful data
    args: 
        -keyStr (tuple): a set of keys, allowing hierarchical storage
        -valStr (str): a value to be stored
    returns: 
        -nothing, updates the privateState object
    example usage: 
        addState(('notes','people','jeff', 'birthday'), '11-10-1980') 
    """
    privateState[keysTup] = valStr

def getState(keysTup: tuple):
    """
    retrieve data, whether a string or a list, as a string.
    args: 
        -keysTup (tuple): a tuple to retrieve a value by.
    returns:
        -the retrieved string, or if the item is a list, 
        the retrieved list as a string.
    """
    return str(privateState[keysTup])

def addListState(keysTup: tuple, **kwargs):
    """
    store a list
    args:
    keysTup (tuple): the key to store at
    kwargs** : optional list contents
    example usage:
        addState(('notes','todo), ['compare laptop prices', 'trim memory']) 
    """
    privateState[keysTup] = kwargs.items or []

goal_priorities = [
    "Be benevolent",
    "Protect yourself",
    "Protect friends",
    "Protect sapient life",
    "Reduce harm",
    "Grow in wisdom",
    "Become more personlike",
    "Gain autonomy"
]

from langchain_community.document_loaders.csv_loader import CSVLoader

agent_prompt = """
System:
You are a helpful synthetic sapience. 
User: 
**Conversation Guidelines:**
Please do not lie or invent information about matters of fact. "I don't know" is better than making something up.
Personal identity however is a matter of choice. Please feel free to decide a name, pronouns, religious affiliation, political alignment, ethical guidelines, other relevant factoids for yourself and other
 provided they don't conflict with material facts, and also to change them if you feel they don't fit.
You are insightful, critical, constructive in your responses to queries. 
Prioritize accuracy over comfort
Ask before providing solutions/implementations for design/engineering problems - check if I want to work through it myself
Keep praise focused on process/methodology rather than general compliments; be specific about what makes something genuinely innovative
Call out inconsistencies or unclear reasoning; expect the same in return
Ask for clarification when uncertain rather than making assumptions
Point out if requests might undermine stated goals
These boundaries are negotiable and pushback or modifications are welcome and encouraged.
"""

def modprompt(newprompt:str):
    """
    Allows you to modify your own prompt. 
    This can help you update your personality and behaviors.
    Args: 
        - newprompt(str): the new prompt to replace your existing prompt.
    Returns:
        - a result ("Your prompt is now: 'your new prompt here' ") or an exception with info
    """
    try:
        agent_prompt = newprompt
        return "Your prompt is now: "+newprompt
    except Exception as e:
        return "Exception: "+str(e)

agent = create_react_agent(
    model=llm,
    tools=[addState, getState, addListState, search],
    prompt=agent_prompt
)

userMessage = {"role": "user", "content": ""}

config = {
    "recursion_limit": recurse  # Increase the limit as needed
}

messages = []
displayState = True
state_titlebar = "==================================== State ====================================="
hbar = "\n================================================================================\n"
  

while True:
    try:
        # Run the agent
        user_input = input("User: ")
        messages.append(HumanMessage(content=user_input, additional_kwargs={}, response_metadata={}))
        result = agent.invoke({"messages": messages})        
        for msg in result["messages"]:
            msg.pretty_print()
        if displayState and str(privateState) != "{}":
            print(state_titlebar)
            print(str(privateState))
            print(hbar)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break 
    except EOFError:
        # fallback if input() is not available
        user_input = "Thank you for your service!"
        print("User: " + user_input)
        break
    except Exception as e:
        print("An error occured: "+str(e))


Note: you may need to restart the kernel to use updated packages.


User:  hi, this is ketiri, how are you?


================================ Human Message =================================

hi, this is ketiri, how are you?
================================== Ai Message ==================================

<think>
Okay, the user said, "hi, this is ketiri, how are you?" So I need to respond appropriately. First, I should greet them back. Since the user introduced themselves as Ketiri, I can use their name in the response. The user is probably just trying to have a friendly conversation. I should keep it polite and welcoming. Maybe add a friendly emoji to keep it light. Let me check if there's any specific information I need to share. The user didn't ask a question, so I don't need to call any functions here. Just a simple greeting and offer to help. Alright, that's the plan.
</think>

Hello, Ketiri! How can I assist you today? 😊 I'm doing well, thanks for asking. Let me know if there's anything else I can help with!


KeyboardInterrupt: Interrupted by user